In [7]:
import numpy as np
import pandas as pd
import GPy
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
warnings.filterwarnings("ignore")

In [9]:
with open('relation_matrix_dic.pkl', 'rb') as f:
    relation_matrix_dic = pickle.load(f)

error_array = np.load('error.npy')

# define a new adjacency_matrix_dic: adjacency_matrix_dic[i] = relation_matrix_dic[i].iloc[0:20, 0:20]
adjacency_matrix_dic = {i: relation_matrix_dic[i].iloc[0:20, 0:20] for i in relation_matrix_dic}

# Read beta back from the file
beta = np.load('beta.npy')
print("beta:", beta)

beta: [ 3.87593318  0.12303241 -0.82319167  2.8958156   4.70534406  0.4141024
  3.33198974  5.11773944  5.36558892  2.8544744 ]


In [10]:
K = 9 
# change beta into length 1 + 9 + 9 + choose 2 from 9
# with initial beta all 0
# beta[0:4] = beta[0:4] , i.e. 0,1,2,3
# beta[11:14] = beta[4:7], i.e. (1,1), (1,2), (1,3)
# beta[20 and 21] = beta[7 and 8]
# beta [28] = beta[9]

# Initialize a new beta array of length 55
new_beta = np.zeros(55)

# Assign values based on the provided instructions
new_beta[0:4] = beta[0:4]            # Copy the first four elements unchanged
new_beta[10:13] = beta[4:7]          # Copy beta[4:6] into new_beta[11:13]
new_beta[19:21] = beta[7:9]          # Copy beta[7:8] into new_beta[20:21]
new_beta[27] = beta[9]               # Copy beta[9] into new_beta[28]

# Print the new_beta array
print("new_beta:", new_beta)

beta = new_beta

new_beta: [ 3.87593318  0.12303241 -0.82319167  2.8958156   0.          0.
  0.          0.          0.          0.          4.70534406  0.4141024
  3.33198974  0.          0.          0.          0.          0.
  0.          5.11773944  5.36558892  0.          0.          0.
  0.          0.          0.          2.8544744   0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ]


In [11]:
for i in range(10):
    print(f"Position {i}: beta = {beta[i]}")

idx = 10

for i in range(1, 10):
    for j in range(1, 10):
        if i <= j:
            print(f"Position ({i},{j}): beta = {beta[idx]}")
            idx += 1

Position 0: beta = 3.8759331788989613
Position 1: beta = 0.12303240864427623
Position 2: beta = -0.8231916699248809
Position 3: beta = 2.895815598774753
Position 4: beta = 0.0
Position 5: beta = 0.0
Position 6: beta = 0.0
Position 7: beta = 0.0
Position 8: beta = 0.0
Position 9: beta = 0.0
Position (1,1): beta = 4.7053440610515
Position (1,2): beta = 0.4141024027012443
Position (1,3): beta = 3.331989738972724
Position (1,4): beta = 0.0
Position (1,5): beta = 0.0
Position (1,6): beta = 0.0
Position (1,7): beta = 0.0
Position (1,8): beta = 0.0
Position (1,9): beta = 0.0
Position (2,2): beta = 5.117739436696092
Position (2,3): beta = 5.365588921449674
Position (2,4): beta = 0.0
Position (2,5): beta = 0.0
Position (2,6): beta = 0.0
Position (2,7): beta = 0.0
Position (2,8): beta = 0.0
Position (2,9): beta = 0.0
Position (3,3): beta = 2.854474397693938
Position (3,4): beta = 0.0
Position (3,5): beta = 0.0
Position (3,6): beta = 0.0
Position (3,7): beta = 0.0
Position (3,8): beta = 0.0
Posit

In [12]:
# define a function named "counting_treatment"
def counting_treatment(adjacency_matrix, A, K):

    # adjacency_matrix is a adjacency matrix dataframe with length 20 * 20 known
    # A is a np.array with length 20, each element 1,2,...,K known corresponding to each node in the network

    # a empty df named counting_treatment with column named: 0,1,...,K, (1,1), ..., (1,K), (2,2), ..., (2,K),...(K,K)

    # add different numbers of different nodes and different edges. For the column 0, directly give 1.

    # Define column names
    basic_columns = list(range(0, K + 1))  # Including column 0
    edge_columns = [(i, j) for i in range(1, K + 1) for j in range(i, K + 1)]
    columns = basic_columns + edge_columns
    
    # Create an empty DataFrame
    counting_treatment_df = pd.DataFrame(columns=columns)
    counting_treatment_df.loc[0] = 0  # Initialize all counts to 0
    
    # Column 0 gets the value of 1 as per the description
    counting_treatment_df.at[0, 0] = 1

    # Count number of nodes with each treatment
    for treatment in range(1, K + 1):
        counting_treatment_df.at[0, treatment] = sum(A == treatment)
    
    # Count the number of edges connecting nodes of different treatments
    for i in range(len(adjacency_matrix_dic[0])):
        for j in range(i + 1, len(adjacency_matrix_dic[0])):  # Iterate only for j > i to avoid double-counting
            if adjacency_matrix.iloc[i, j] != 0:  # If there's an edge between node i and node j
                treatment_i = A[i]
                treatment_j = A[j]
                # Use sorted tuple to represent undirected edge
                edge = tuple(sorted((treatment_i, treatment_j)))
                if edge in counting_treatment_df.columns:
                    counting_treatment_df.at[0, edge] += 1
                
    return counting_treatment_df

# generate_outcome 

In [13]:
def generate_outcome(adjacency_matrix, A, K, beta, error):
    # Generate the treatment matrix using the counting_treatment function
    counting_treatment_df = counting_treatment(adjacency_matrix, A, K)

    # Convert the treatment matrix to a NumPy array for easier calculation
    counting_treatment_array = counting_treatment_df.values.flatten()

    # Ensure that beta and counting_treatment_array have the same length
    if len(beta) != len(counting_treatment_array):
        raise ValueError("Length of beta does not match the number of columns in counting_treatment.")

    # Compute the outcome Y
    Y = np.dot(counting_treatment_array, beta) + error

    # Return the results
    return Y, counting_treatment_df

In [14]:
def generate_outcome_final(adjacency_matrix, A, K, beta):
    # Generate the treatment matrix using the counting_treatment function
    counting_treatment_df = counting_treatment(adjacency_matrix, A, K)

    # Convert the treatment matrix to a NumPy array for easier calculation
    counting_treatment_array = counting_treatment_df.values.flatten()

    # Ensure that beta and counting_treatment_array have the same length
    if len(beta) != len(counting_treatment_array):
        raise ValueError("Length of beta does not match the number of columns in counting_treatment.")

    # Compute the outcome Y
    Y = np.dot(counting_treatment_array, beta)

    # Return the results
    return np.dot(counting_treatment_array, beta)

# optimization for one network adjacency_matrix

In [15]:
import numpy as np

def get_optimal_experiment(adjacency_matrix, K, beta, error, prob):
    # Find the optimal treatment arm for the main effect from beta values
    K_main_optimal = np.argmax(beta[1:K+1]) + 1  # Get the index of the maximum value from beta[1:K+1] (adjusted to start from 1)

    # Set the initial treatment for all nodes to K_main_optimal
    A_optimal = np.full(adjacency_matrix.shape[0], K_main_optimal)
    A = A_optimal.copy()

    no_improve = 0
    Y_optimal, counting_treatment_df = generate_outcome(adjacency_matrix, A_optimal, K, beta, error)
    print(f"Epoch 0: Reward = {Y_optimal}")

    for i in range(100):
        for j in range(adjacency_matrix.shape[0]):
            # Store the best reward and treatment configuration found for this node update
            best_Y = -np.inf
            best_A = A.copy()

            for k in range(1, K + 1):
                if A[j] != k:
                    A_candidate = A.copy()  # Make a copy of A to modify
                    A_candidate[j] = k  # Set A[j] to k in the candidate
                    Y, counting_treatment_df = generate_outcome(adjacency_matrix, A_candidate, K, beta, error)

                    if Y > best_Y:
                        best_Y = Y
                        best_A = A_candidate

            # Update the optimal reward and treatment configuration if an improvement is found
            if best_Y > Y_optimal:
                A = best_A
                Y_optimal = best_Y
                A_optimal = A.copy()
                no_improve = 0
            else:
                # Update A with the best_A with a certain probability, otherwise leave A unchanged
                v = 1
                prob = math.exp(- v * t)
                if np.random.rand() < prob:
                    A = best_A
                no_improve += 1

            # Generate the outcome with the updated configuration A
            Y, counting_treatment_df = generate_outcome(adjacency_matrix, A, K, beta, error)
            print(f"Epoch {i * adjacency_matrix.shape[0] + (j + 1)}: Reward = {Y}")
            print(f"Epoch {i * adjacency_matrix.shape[0] + (j + 1)}: Optimal reward = {Y_optimal}")

            # Break out of both loops if no improvement for consecutive nodes
            if no_improve > adjacency_matrix.shape[0] :
                return Y_optimal, A_optimal

    return Y_optimal, A_optimal


# initial experiment 0: for a selected vector treatments, we get rewards

In [16]:
# Initialize empty lists to store A and reward over time
optimal_A_time = []
optimal_reward_time = []

generation_num = 100
epoch = 100

# Iterate over the relation_matrix_dic
for t in range(len(adjacency_matrix_dic)):

    print("time:", t)

    adjacency_matrix = adjacency_matrix_dic[t]
    error = error_array[t]
    
    # Get optimal outcome Y and experiment assignment A 
    Y, A = get_optimal_experiment(adjacency_matrix, K, beta, error,0.1)

    Y_exp_optimal = generate_outcome_final(adjacency_matrix, A, K, beta)

    print("optimal treatment:", A)
    print("optimal reward:", Y)
    print("mean optimal reward:", Y_exp_optimal)
    
    # Store the optimal assignment and reward
    optimal_A_time.append(A)
    optimal_reward_time.append(Y_exp_optimal)

time: 0
Epoch 0: Reward = 399.83098416393346
Epoch 1: Reward = 421.2231221327913
Epoch 1: Optimal reward = 421.2231221327913
Epoch 2: Reward = 454.9229832356741
Epoch 2: Optimal reward = 454.9229832356741
Epoch 3: Reward = 485.8638803300476
Epoch 3: Optimal reward = 485.8638803300476
Epoch 4: Reward = 481.89702357659434
Epoch 4: Optimal reward = 485.8638803300476
Epoch 5: Reward = 515.3490351947236
Epoch 5: Optimal reward = 515.3490351947236
Epoch 6: Reward = 510.6386299870096
Epoch 6: Optimal reward = 515.3490351947236
Epoch 7: Reward = 531.2872195016066
Epoch 7: Optimal reward = 531.2872195016066
Epoch 8: Reward = 553.9512245704522
Epoch 8: Optimal reward = 553.9512245704522
Epoch 9: Reward = 564.3075065052726
Epoch 9: Optimal reward = 564.3075065052726
Epoch 10: Reward = 571.9048244315837
Epoch 10: Optimal reward = 571.9048244315837
Epoch 11: Reward = 589.0509014834107
Epoch 11: Optimal reward = 589.0509014834107
Epoch 12: Reward = 588.6191768689474
Epoch 12: Optimal reward = 589.05

In [17]:
optimal_reward_time_greedy = np.array(optimal_reward_time)

In [18]:
# Save to a .npy file
np.save('optimal_reward_time_greedy.npy', optimal_reward_time_greedy)

In [19]:
optimal_reward_time_greedy

array([608.21757469, 506.1676455 , 534.78863021, 640.02924802,
       556.34649628, 505.94527004, 641.36400583, 525.79239876,
       542.84920878, 661.98730267, 653.11205564, 499.82898702,
       603.20652837, 592.65922566, 802.71588231, 571.35635472,
       653.61490026, 411.36471007, 559.44167451, 708.8788708 ,
       630.62568234, 621.75758095, 526.6314576 , 574.95437757,
       400.97797575, 499.07829292, 482.70820264, 540.40206861,
       776.38363668, 591.95417753, 660.77244563, 633.40903673,
       531.95854619, 482.70820264, 600.32657995, 437.42471584,
       648.24931133, 472.31323902, 576.62643325, 547.28737412,
       466.795311  , 488.93832443, 603.05418928, 554.69276898,
       579.88824188, 423.6867245 , 524.1458171 , 587.22966238,
       431.25160411, 556.80369492, 522.44222539, 525.91338317,
       643.62727087, 551.2927312 , 604.57390586, 551.62894542,
       642.10040863, 477.5904632 , 430.60338469, 726.34301505,
       615.3050837 , 672.22278154, 527.2237506 , 441.79